# Preprocessing phase

In [ ]:
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split

In [ ]:
dataset = pd.read_csv('../Data/export.csv')

In [ ]:
columnsName = ['db_sc', 'id_sc', 'pv', 'data', 'cassa', 'cassiere', 'num_scontrino', 'ora', 'tessera', 't_flag', 'num_riga',
                'r_reparto_cdaplus', 'r_ean', 'r_qta_pezzi', 'r_peso', 'r_importo_lordo', 'r_imponibile', 'r_iva', 'r_sconto',
                'r_sconto_fide', 'r_sconto_rip', 'r_tipo_riga', 'cod_prod', 'descr_prod', 'cat_mer', 'cod_forn', 'descr_forn',
                'liv1', 'descr_liv1', 'liv2', 'descr_liv2', 'liv3', 'descr_liv3', 'liv4', 'descr_liv4', 'liv5', 'descr_liv5',
                'liv6', 'descr_liv6', 'tipologia', 'descr_tipologia', 'cod_rep', 'descr_rep']

dataset.columns = columnsName

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
dataset

In [ ]:
dataset['cod_prod'] = dataset['cod_prod'].astype(str)
dataset['cod_prod'] = dataset['cod_prod'].apply(lambda x: x.split('.')[0] if '.0' in x else x)


In [ ]:
dataset = dataset.dropna(subset=['cod_prod'])
dataset = dataset[dataset['r_tipo_riga'] != 'ANNULLO']
dataset = dataset[dataset['r_importo_lordo'] != 0]
dataset = dataset[dataset['cat_mer'] != 'NNNNN']
dataset = dataset[dataset['cod_prod'] != '1090011'] # shopping bag

For implement a Recommendation system, we must know who is the client.

The only feature that return this information is 'tessera' but the fidelity customers are much less.

In [ ]:
dataset = dataset[dataset['tessera'].notna()]

In [ ]:
dataset['data'] = pd.to_datetime(dataset['data'])
dataset['ora'] = pd.to_datetime(dataset['ora'], format='%H:%M').dt.time

In [ ]:
df = dataset[['id_sc', 'pv', 'data', 'ora', 'tessera', 'r_qta_pezzi', 'r_peso', 'r_importo_lordo', 'r_sconto', 'r_sconto_rip', 'cod_prod', 'descr_prod',
                'cat_mer', 'cod_forn', 'liv1', 'descr_liv1', 'liv2', 'descr_liv2', 'liv3', 'descr_liv3', 'liv4', 'descr_liv4']]

In [ ]:
df_GenMar = df[(df['data'].dt.month >= 1) & (df['data'].dt.month <= 3)][['tessera', 'cod_prod', 'descr_prod']]
df_AprGiu = df[(df['data'].dt.month >= 4) & (df['data'].dt.month <= 6)][['tessera', 'cod_prod', 'descr_prod']]
df_LugSet = df[(df['data'].dt.month >= 7) & (df['data'].dt.month <= 9)][['tessera', 'cod_prod', 'descr_prod']]
df_OttDic = df[(df['data'].dt.month >= 10) & (df['data'].dt.month <= 12)][['tessera', 'cod_prod', 'descr_prod']]

In [ ]:
def preprocess(df):
    return df.groupby(['tessera', 'cod_prod', 'descr_prod']).agg(value=('cod_prod', 'count')).reset_index()

In [ ]:
df_GenMar = preprocess(df_GenMar)
df_AprGiu = preprocess(df_AprGiu)
df_LugSet = preprocess(df_LugSet)
df_OttDic = preprocess(df_OttDic)

In [ ]:
np.random.seed(42)
random.seed(42)

In [ ]:
train_data_GenMar, test_data_GenMar = train_test_split(df_GenMar, test_size=0.25)
train_data_AprGiu, test_data_AprGiu = train_test_split(df_AprGiu, test_size=0.25)
train_data_LugSet, test_data_LugSet = train_test_split(df_LugSet, test_size=0.25)
train_data_OttDic, test_data_OttDic = train_test_split(df_OttDic, test_size=0.25)

In [ ]:
train_data_GenMar.to_csv('../Data/train_data_GenMar.csv', index=False)
train_data_AprGiu.to_csv('../Data/train_data_AprGiu.csv', index=False)
train_data_LugSet.to_csv('../Data/train_data_LugSet.csv', index=False)
train_data_OttDic.to_csv('../Data/train_data_OttDic.csv', index=False)

test_data_GenMar.to_csv('../Data/test_data_GenMar.csv', index=False)
test_data_AprGiu.to_csv('../Data/test_data_AprGiu.csv', index=False)
test_data_LugSet.to_csv('../Data/test_data_LugSet.csv', index=False)
test_data_OttDic.to_csv('../Data/test_data_OttDic.csv', index=False)